In [1]:
from pyspark.sql import SparkSession

In [7]:
spark = SparkSession.builder.appName("STRUCTURED_DF").master("local[8]").getOrCreate()

In [8]:
spark

In [9]:
df = spark.read.text("/home/guilhermefmk/Documentos/labs_spark/data/kddcup.data")

In [13]:
df2 = spark.read.csv("/home/guilhermefmk/Documentos/labs_spark/data/kddcup.data", sep=',')

In [14]:
df2.printSchema()

root
 |-- _c0: string (nullable = true)
 |-- _c1: string (nullable = true)
 |-- _c2: string (nullable = true)
 |-- _c3: string (nullable = true)
 |-- _c4: string (nullable = true)
 |-- _c5: string (nullable = true)
 |-- _c6: string (nullable = true)
 |-- _c7: string (nullable = true)
 |-- _c8: string (nullable = true)
 |-- _c9: string (nullable = true)
 |-- _c10: string (nullable = true)
 |-- _c11: string (nullable = true)
 |-- _c12: string (nullable = true)
 |-- _c13: string (nullable = true)
 |-- _c14: string (nullable = true)
 |-- _c15: string (nullable = true)
 |-- _c16: string (nullable = true)
 |-- _c17: string (nullable = true)
 |-- _c18: string (nullable = true)
 |-- _c19: string (nullable = true)
 |-- _c20: string (nullable = true)
 |-- _c21: string (nullable = true)
 |-- _c22: string (nullable = true)
 |-- _c23: string (nullable = true)
 |-- _c24: string (nullable = true)
 |-- _c25: string (nullable = true)
 |-- _c26: string (nullable = true)
 |-- _c27: string (nullable = tru

In [10]:
df.printSchema()

root
 |-- value: string (nullable = true)



In [11]:
df.show()

+--------------------+
|               value|
+--------------------+
|0,tcp,http,SF,215...|
|0,tcp,http,SF,162...|
|0,tcp,http,SF,236...|
|0,tcp,http,SF,233...|
|0,tcp,http,SF,239...|
|0,tcp,http,SF,238...|
|0,tcp,http,SF,235...|
|0,tcp,http,SF,234...|
|0,tcp,http,SF,239...|
|0,tcp,http,SF,181...|
|0,tcp,http,SF,184...|
|0,tcp,http,SF,185...|
|0,tcp,http,SF,239...|
|0,tcp,http,SF,181...|
|0,tcp,http,SF,236...|
|0,tcp,http,SF,233...|
|0,tcp,http,SF,238...|
|0,tcp,http,SF,235...|
|0,tcp,http,SF,234...|
|0,tcp,http,SF,239...|
+--------------------+
only showing top 20 rows



In [12]:
# Split data (if needed)

from pyspark.sql.functions import split

split_col = split(df['value'], ',')
df = df.withColumn('Protocol', split_col.getItem(1)) \
       .withColumn('Service', split_col.getItem(2)) \
       .withColumn('flag', split_col.getItem(3)) \
       .withColumn('src_bytes', split_col.getItem(4)) \
       .withColumn('dst_bytes', split_col.getItem(5)) \
       .withColumn('urgent', split_col.getItem(8)) \
       .withColumn('num_failed_logins', split_col.getItem(10)) \
       .withColumn('root_shell', split_col.getItem(13)) \
       .withColumn('guest_login', split_col.getItem(21)) \
       .withColumn('label', split_col.getItem(41)) \
       .drop('value')

df.show()

+--------+-------+----+---------+---------+------+-----------------+----------+-----------+-------+
|Protocol|Service|flag|src_bytes|dst_bytes|urgent|num_failed_logins|root_shell|guest_login|  label|
+--------+-------+----+---------+---------+------+-----------------+----------+-----------+-------+
|     tcp|   http|  SF|      215|    45076|     0|                0|         0|          0|normal.|
|     tcp|   http|  SF|      162|     4528|     0|                0|         0|          0|normal.|
|     tcp|   http|  SF|      236|     1228|     0|                0|         0|          0|normal.|
|     tcp|   http|  SF|      233|     2032|     0|                0|         0|          0|normal.|
|     tcp|   http|  SF|      239|      486|     0|                0|         0|          0|normal.|
|     tcp|   http|  SF|      238|     1282|     0|                0|         0|          0|normal.|
|     tcp|   http|  SF|      235|     1337|     0|                0|         0|          0|normal.|


In [16]:
df = df.repartition(10)
print(df.rdd.getNumPartitions())

df.createOrReplaceTempView("kcup")

10


In [17]:
df.groupBy("label").count().orderBy('count', ascending=False).show()

+----------------+-------+
|           label|  count|
+----------------+-------+
|          smurf.|2807886|
|        neptune.|1072017|
|         normal.| 972781|
|          satan.|  15892|
|        ipsweep.|  12481|
|      portsweep.|  10413|
|           nmap.|   2316|
|           back.|   2203|
|    warezclient.|   1020|
|       teardrop.|    979|
|            pod.|    264|
|   guess_passwd.|     53|
|buffer_overflow.|     30|
|           land.|     21|
|    warezmaster.|     20|
|           imap.|     12|
|        rootkit.|     10|
|     loadmodule.|      9|
|      ftp_write.|      8|
|       multihop.|      7|
+----------------+-------+
only showing top 20 rows



In [18]:
sql_query = """ SELECT Protocol,
                CASE label
                  WHEN 'normal.' THEN 'no attack'
                  ELSE 'attack'
                END AS State,
                COUNT(*) as freq
              FROM kcup
              WHERE guest_login != 1
              GROUP BY Protocol, State
              ORDER BY Protocol DESC
            """

spark.sql(sql_query).show()

+--------+---------+-------+
|Protocol|    State|   freq|
+--------+---------+-------+
|     udp|   attack|   2940|
|     udp|no attack| 191348|
|     tcp|no attack| 764894|
|     tcp|   attack|1101613|
|    icmp|   attack|2820782|
|    icmp|no attack|  12763|
+--------+---------+-------+

